## Setup

In [1]:
import pprint
import networkx as nx
import random
import time
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np

from planners.IPBasicPRM import BasicPRM
from optimize_path import OptimizeFlyby
from planners.IPPerfMonitor import IPPerfMonitor
from collision_checker import CollisionChecker
from benchmarks import construct_benchmark_environments
from analysis import (
    create_interactive_viewer,
    interactive_radius_exploration,
    render_initial_path,
    retrieve_path_positions,
    calculate_path_length,
    plot_performance_data,
    clear_graph_attributes
)

## Load Environments

In [2]:
env_dict = construct_benchmark_environments()

## Create Planner and Collision Checker for each environment

In [3]:
for name, item in env_dict.items():
    # 1. Initialize a CollisionChecker with the environment
    cc = CollisionChecker(item["env"])

    # 2. Initialize a Planner with the collisionChecker
    planner = BasicPRM(cc)

    # 3. Construct the NetworkX Graph from the predefined solution path
    G = nx.Graph()

    node_names = []

    for i, coord in enumerate(item["smooth_path"]):
        # Determine the name based on position in the list
        if i == 0:
            n = "start"
        elif i == len(item["smooth_path"]) - 1:
            n = "goal"
        else:
            n = f"{i}" # Naming middle nodes generically
        
        node_names.append(n)
        
        # Add node with the 'pos' attribute
        G.add_node(n, pos=coord)

    # Add edges connecting each node to the next
    for i in range(len(node_names) - 1):
        current_node = node_names[i]
        next_node = node_names[i+1]
        G.add_edge(current_node, next_node)

    # 4. Inject the constructed graph into the Planner instance
    planner.graph = G
    
    # 5. Add the node names and the planner to the environment dictionary
    env_dict[name]["solution_node_names"] = node_names
    env_dict[name]["planner"] = planner
    del env_dict[name]["smooth_path"]

# pprint.pprint(env_dict)

## Show the Environments

In [4]:
# 1. Define the worker function
def render_env(Env):
    item = env_dict[Env]
    planner = item["planner"]
    
    ax = planner._collisionChecker.draw_enviroments()
    path_pos = retrieve_path_positions(planner.graph, item['solution_node_names'])
    planner._collisionChecker.draw_path(path_pos, ax=ax)
    plt.title(f"Environment {Env}")
    plt.show()

# 2. Call the generic viewer with 1 parameter
config = {'Env': list(env_dict.keys())}
create_interactive_viewer(config, render_env)

## Round the corners symmetrically

In [ ]:
optimizer = OptimizeFlyby()

interactive_radius_exploration(env_dict, optimizer)

## Global k Optimization

In [ ]:
optimizer = OptimizeFlyby()

for name, item in env_dict.items():
    print(f"\nProcessing Environment {name}...")
    
    # 1. Setup
    planner = item['planner']
    node_names = item['solution_node_names']
    IPPerfMonitor.clearData()

    # 2. Run Optimization
    optimized_path = optimizer.analyze_optimal_k(planner, node_names, r_fixed=0.5)

    # 3. Get performance data
    df = IPPerfMonitor.dataFrame()

    # Extract number of collisionChecks and corresponding calculation time
    funcs = ['lineInCollision', 'curveInCollision', 'pointInCollision']
    stats = {}
    for f in funcs:
        f_data = df[df['name'] == f]
        if not f_data.empty:
            stats[f] = {'count': len(f_data), 'time': f_data['time'].sum()}
        else:
            stats[f] = {'count': 0, 'time': 0.0}
    
    # 4. Calculate path lengths
    len_original = calculate_path_length(planner, node_names, use_curves=False)
    len_optimized = calculate_path_length(planner, node_names, use_curves=True)

    # 5. Visualizations
    ax = item['planner']._collisionChecker.draw_enviroments()
    item['planner']._collisionChecker.draw_optimized_path(optimized_path, item["planner"], ax)
    plt.show()

    plot_performance_data(stats, len_original, len_optimized)

## Optimize individual k's

In [ ]:
optimizer = OptimizeFlyby()

for name, item in env_dict.items():
    print(f"\n{'='*60}")
    print(f"Processing Environment {name} (Individual Corner Optimization)")
    print(f"{'='*60}")
    
    # 1. Setup
    planner = item['planner']
    node_names = item['solution_node_names']
    
    # 2. Preparation: Clean slate
    clear_graph_attributes(planner)
    IPPerfMonitor.clearData()

    # 3. Run Optimization (Coordinate Descent)
    # This function repeatedly calls optimizePath to test k values for each node individually.
    # It returns the final result with the best configuration found.
    optimized_path = optimizer.optimize_individual_corners(node_names, planner, config={'r_init': 0.5})

    # 4. Get Performance Data
    df = IPPerfMonitor.dataFrame()

    # Extract collision check stats
    funcs = ['lineInCollision', 'curveInCollision', 'pointInCollision']
    stats = {}
    for f in funcs:
        f_data = df[df['name'] == f]
        if not f_data.empty:
            stats[f] = {'count': len(f_data), 'time': f_data['time'].sum()}
        else:
            stats[f] = {'count': 0, 'time': 0.0}
    
    # 5. Calculate Path Lengths
    len_original = calculate_path_length(planner, node_names, use_curves=False)
    len_optimized = calculate_path_length(planner, node_names, use_curves=True)

    # 6. Visualizations
    
    # A. Optimized Path Visualization
    ax = item['planner']._collisionChecker.draw_enviroments()
    
    # Draw the path (this will show specific k values for each node)
    item['planner']._collisionChecker.draw_optimized_path(optimized_path, item["planner"], ax)
    
    plt.title(f"Environment {name}: Individual Corner Optimization")
    plt.show() 

    # B. Performance Metrics Chart
    plot_performance_data(stats, len_original, len_optimized)